In [82]:
library('tidyverse')
library('robustbase')
library('corrplot')
library('RColorBrewer')

In [126]:
master <- "/Users/laurituominen/Documents/Research/Reettis/old_new_analyses/master/"
dat <- read_csv(paste0(master, 'merged_data.csv'),show_col_types = FALSE)

dat <- dat %>% mutate(Sex = factor(Sex, levels = c('M', 'F')),
                      group =factor(Group, levels=c('CTR', 'CHR', 'FEP')))

# exposure vs cortical thickness 
model <- dat %>% filter(group %in% c('FEP', 'CHR')) %>% 
  lmrob(mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group, na.action=na.omit, data=.)

model %>% summary()


Call:
lmrob(formula = mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group, 
    data = ., na.action = na.omit)
 \--> method = "MM"
Residuals:
      Min        1Q    Median        3Q       Max 
-0.193423 -0.060353 -0.002418  0.060400  0.197570 

Coefficients:
                       Estimate Std. Error t value Pr(>|t|)    
(Intercept)           2.636e+00  3.093e-02  85.218  < 2e-16 ***
Lifetime_ap_exposure -1.128e-06  2.720e-07  -4.145 6.19e-05 ***
Age                  -2.814e-03  1.070e-03  -2.630   0.0096 ** 
SexF                 -3.191e-02  1.444e-02  -2.211   0.0289 *  
GroupFEP             -1.975e-02  1.527e-02  -1.294   0.1982    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Robust residual standard error: 0.08595 
Multiple R-squared:  0.183,	Adjusted R-squared:  0.1571 
Convergence in 9 IRWLS iterations

Robustness weights: 
 15 weights are ~= 1. The remaining 116 ones are summarized as
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5765  0.8926 

In [127]:
model %>% confint()

print(paste('residuals:', model$df.residual))

,2.5 %,97.5 %
(Intercept),2.574810e+00,2.697239e+00
Lifetime_ap_exposure,-1.665802e-06,-5.892332e-07
Age,-4.931425e-03,-6.968154e-04
SexF,-6.047924e-02,-3.345109e-03
GroupFEP,-4.995567e-02,1.046238e-02


[1] "residuals: 126"


In [128]:
# covariates in sensitivity analyses 
covars <- c("Total_symptoms_score", "Positive_symptoms_score", "Negative_symptoms_score", "BMI", "Hospital_days", "Times_admitted",
            "GAF", "SOFAS")

# estimate the effects including one of the sensitivity covariates in the model at the time 
vals <- matrix(nrow=length(covars), ncol=7 )
for (i in seq(from=1,to=length(covars))){
  formula <- reformulate(termlabels =  c("Lifetime_ap_exposure","Age","Sex","Group", covars[i]), response = 'mean_thickness')
  m <- dat %>% filter(group %in% c('FEP', 'CHR')) %>%
    lmrob(formula = formula, data=.)
  vals[i,]<- c(m$df.residual, as.vector(summary(m)$coefficients[2,]), as.vector(confint(m)[2,])) # get parameters for "Lifetime_ap_exposure"   
}

# make printable 
df <- as.data.frame(vals)
df$covariate <- covars 
df <- df[,c(8,2,6,7,1, 4,5)]
names(df) <- c("Covariate", "Estimate", "2.5% quantile", "97.5% quantile", "df", "t-value", "p-value")


In [129]:
# do some rounding
round_sci <- function(x, digits) {
  format(as.numeric(x), scientific = TRUE, digits = digits)
}

rounded_df <- df %>%
  mutate_at(vars('Estimate','2.5% quantile', '97.5% quantile', 'p-value'), list(~ round_sci(., digits = 3))) %>%
  mutate_at(vars('t-value'), list(~ round(., digits=3)))

rounded_df
write_csv(rounded_df, '/Users/laurituominen/Documents/Research/Reettis/neuromaps/tables/stable1_sensitivity analyses.csv')

Covariate,Estimate,2.5% quantile,97.5% quantile,df,t-value,p-value
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
Total_symptoms_score,-1.20e-06,-1.69e-06,-6.99e-07,120,-4.766,5.33e-06
Positive_symptoms_score,-1.20e-06,-1.72e-06,-6.84e-07,120,-4.598,1.06e-05
Negative_symptoms_score,-1.21e-06,-1.70e-06,-7.15e-07,120,-4.856,3.64e-06
BMI,-1.09e-06,-1.59e-06,-5.87e-07,125,-4.295,3.47e-05
Hospital_days,-1.04e-06,-1.66e-06,-4.18e-07,124,-3.309,1.23e-03
Times_admitted,-1.05e-06,-1.61e-06,-4.91e-07,124,-3.717,3.04e-04
GAF,-1.16e-06,-1.72e-06,-6.04e-07,125,-4.119,6.87e-05
SOFAS,-1.16e-06,-1.71e-06,-6.14e-07,125,-4.186,5.32e-05


## plot correlations between confounders 

In [130]:
# Use a Brewer palette and invert it
brewer_palette <- brewer.pal(11, "RdBu")
inverted_brewer_palette <- rev(brewer_palette)

covariates <- dat[covars]
names(covariates) <- c('Total', 'Positive', 'Negative', 'BMI', 'Hosp days', 'Times admitted', 'GAF', 'SOFAS')
correlation <- cor(covariates, use = "pairwise.complete.obs")
png('/Users/laurituominen/Documents/Research/Reettis/neuromaps/figures/Correlations_between_confounders_in_Turku_sample.png',width = 5, units = 'in', height=5, res = 300)

corrplot(correlation, method="ellipse", col = inverted_brewer_palette,  type = 'lower', diag = FALSE)
# make plot
dev.off()

pdf 
  2

In [131]:
## create a more comprehensive model by hand picking covariates 
# covs to include: Total score, GAF, BMI, Hospital_days 
model2 <- dat %>% filter(group %in% c('FEP', 'CHR')) %>% 
  lmrob(mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group +Total_symptoms_score +BMI + Hospital_days +GAF , na.action=na.omit, data=.)

In [132]:
round_sci <- function(x, digits) {
  format(as.numeric(x), scientific = TRUE, digits = digits)
}

df <- summary(model2)

rounded_coeffs <- as.data.frame(df$coefficients) %>%
  mutate_at(vars('Estimate','Std. Error', 'Pr(>|t|)'), list(~ round_sci(., digits = 3)))%>%
  mutate_at(vars('t value'), list(~ round(., digits=2)))

rounded_coeffs['predictors'] = rownames(rounded_coeffs)
rounded_coeffs <- rounded_coeffs %>% relocate('predictors')
rounded_coeffs

write_csv(rounded_coeffs, '/Users/laurituominen/Documents/Research/Reettis/neuromaps/tables/stable1_sensitivity_analyses_multiple.csv',  col_names = TRUE)



,predictors,Estimate,Std. Error,t value,Pr(>|t|)
,<chr>,<chr>,<chr>,<dbl>,<chr>
(Intercept),(Intercept),2.65e+00,6.78e-02,39.09,1.27e-68
Lifetime_ap_exposure,Lifetime_ap_exposure,-1.10e-06,2.88e-07,-3.80,2.32e-04
Age,Age,-2.97e-03,1.02e-03,-2.90,4.42e-03
SexF,SexF,-3.35e-02,1.45e-02,-2.31,2.29e-02
GroupFEP,GroupFEP,-1.87e-02,1.51e-02,-1.24,2.18e-01
Total_symptoms_score,Total_symptoms_score,7.66e-04,6.79e-04,1.13,2.61e-01
BMI,BMI,-2.10e-03,1.14e-03,-1.85,6.71e-02
Hospital_days,Hospital_days,-1.97e-04,1.59e-04,-1.24,2.17e-01
GAF,GAF,4.39e-04,6.78e-04,0.65,5.19e-01


In [133]:
# create squared effects 
dat$Age2 <- dat$Age^2
dat$Total_symptoms_score2 <- dat$Total_symptoms_score^2 
dat$BMI2 <- dat$BMI^2
dat$Hospital_days2 <- dat$Hospital_days^2 
dat$GAF2 <- dat$GAF^2
dat$Negative_symptoms_score2 <- dat$Negative_symptoms_score^2
dat$Positive_symptoms_score2 <- dat$Positive_symptoms_score^2
dat$SOFAS2 <- dat$SOFAS^2

model3 <- dat %>% filter(group %in% c('FEP', 'CHR')) %>% 
  lmrob(mean_thickness ~ Lifetime_ap_exposure + Age +Age2+ Sex + Group +Total_symptoms_score+Total_symptoms_score2 +BMI + BMI2 + Hospital_days + Hospital_days2 +GAF + GAF2 , na.action=na.omit, data=.)

In [134]:
# print and save the coeffs 
df <- summary(model3)

rounded_coeffs <- as.data.frame(df$coefficients) %>%
  mutate_at(vars('Estimate','Std. Error', 'Pr(>|t|)'), list(~ round_sci(., digits = 3)))%>%
  mutate_at(vars('t value'), list(~ round(., digits=2)))

rounded_coeffs['predictors'] = rownames(rounded_coeffs)
rounded_coeffs <- rounded_coeffs %>% relocate('predictors')
rounded_coeffs

write_csv(rounded_coeffs, '/Users/laurituominen/Documents/Research/Reettis/neuromaps/tables/stable1_sensitivity_analyses_multiple_squared.csv')

,predictors,Estimate,Std. Error,t value,Pr(>|t|)
,<chr>,<chr>,<chr>,<dbl>,<chr>
(Intercept),(Intercept),2.68e+00,2.05e-01,13.11,2.69e-24
Lifetime_ap_exposure,Lifetime_ap_exposure,-9.28e-07,2.92e-07,-3.17,1.94e-03
Age,Age,-1.79e-02,9.31e-03,-1.92,5.78e-02
Age2,Age2,2.47e-04,1.52e-04,1.63,1.06e-01
SexF,SexF,-2.52e-02,1.55e-02,-1.63,1.06e-01
GroupFEP,GroupFEP,-1.16e-02,1.58e-02,-0.73,4.65e-01
Total_symptoms_score,Total_symptoms_score,-2.64e-03,4.13e-03,-0.64,5.24e-01
Total_symptoms_score2,Total_symptoms_score2,4.05e-05,5.13e-05,0.79,4.31e-01
BMI,BMI,8.24e-03,8.24e-03,1.00,3.19e-01


## PCA of the covariates and a scree plot

In [139]:
variables <- c("mean_thickness", "Lifetime_ap_exposure", "Age", "Age2", "Sex", "Group", "Total_symptoms_score", "Total_symptoms_score2",
               "Positive_symptoms_score","Positive_symptoms_score2", "Negative_symptoms_score","Negative_symptoms_score2", "BMI","BMI2",
               "Hospital_days","Hospital_days2", "GAF","GAF2", "SOFAS","SOFAS2")

dat2 <- dat[variables]
dat2 <- dat2 %>% filter(Group %in% c('FEP', 'CHR'))
dat2 <- dat2[complete.cases(dat2), ]

# do PCA on the data 
dat_PCA <- dat2[c("Total_symptoms_score", "Total_symptoms_score2",
               "Positive_symptoms_score","Positive_symptoms_score2", "Negative_symptoms_score","Negative_symptoms_score2", 
                  "BMI","BMI2", "Hospital_days","Hospital_days2", "GAF","GAF2", "SOFAS","SOFAS2")]
pca_result <- prcomp(dat_PCA, center = TRUE, scale. = TRUE)

png('/Users/laurituominen/Documents/Research/Reettis/neuromaps/figures/Screeplot_PCA_confounders_in_Turku_sample.png',width = 5, units = 'in', height=5, res = 300)

plot(pca_result, type = "l", main = "Scree Plot")

# make plot
dev.off()

# add PCA results to dat2 
PCs <- as.data.frame(pca_result$x)
PCs$ID <- 1:nrow(PCs)
dat2$ID <-   1:nrow(dat2)
dat2 <- merge(dat2, PCs)

summary(pca_result)



pdf 
  2

Importance of components:
                          PC1    PC2    PC3    PC4     PC5     PC6     PC7
Standard deviation     2.5772 1.4772 1.4353 1.2840 0.97600 0.44527 0.37615
Proportion of Variance 0.4744 0.1559 0.1472 0.1178 0.06804 0.01416 0.01011
Cumulative Proportion  0.4744 0.6303 0.7775 0.8952 0.96327 0.97743 0.98753
                           PC8     PC9   PC10    PC11    PC12    PC13    PC14
Standard deviation     0.27541 0.22188 0.1449 0.10642 0.09887 0.07123 0.04764
Proportion of Variance 0.00542 0.00352 0.0015 0.00081 0.00070 0.00036 0.00016
Cumulative Proportion  0.99295 0.99647 0.9980 0.99878 0.99948 0.99984 1.00000

## Use first 4 PCs as covariates 

In [140]:

model4 <- dat2 %>% 
  lmrob(mean_thickness ~ Lifetime_ap_exposure + Age + Sex + Group + PC1 + PC2 + PC3 + PC4, na.action=na.omit, data=.)

df <- summary(model4)
rounded_coeffs <- as.data.frame(df$coefficients) %>%
  mutate_at(vars('Estimate','Std. Error', 'Pr(>|t|)'), list(~ round_sci(., digits = 3)))%>%
  mutate_at(vars('t value'), list(~ round(., digits=2)))

rounded_coeffs['predictors'] = rownames(rounded_coeffs)
rounded_coeffs <- rounded_coeffs %>% relocate('predictors')
rounded_coeffs

write_csv(rounded_coeffs, '/Users/laurituominen/Documents/Research/Reettis/neuromaps/tables/stable1_sensitivity_analyses_PCA.csv')

,predictors,Estimate,Std. Error,t value,Pr(>|t|)
,<chr>,<chr>,<chr>,<dbl>,<chr>
(Intercept),(Intercept),2.65e+00,3.03e-02,87.35,1.11e-107
Lifetime_ap_exposure,Lifetime_ap_exposure,-1.14e-06,2.72e-07,-4.20,5.18e-05
Age,Age,-2.95e-03,1.03e-03,-2.86,5.09e-03
SexF,SexF,-3.48e-02,1.46e-02,-2.39,1.83e-02
GroupFEP,GroupFEP,-2.15e-02,1.51e-02,-1.43,1.57e-01
PC1,PC1,-3.49e-04,2.54e-03,-0.14,8.91e-01
PC2,PC2,9.24e-03,4.03e-03,2.30,2.34e-02
PC3,PC3,-7.05e-04,4.02e-03,-0.18,8.61e-01
PC4,PC4,-4.36e-03,5.68e-03,-0.77,4.44e-01
